# LTSM Model for Twitter Sentiment Analysis

***Source idea:***https://www.kaggle.com/stoicstatic/twitter-sentiment-analysis-for-beginners/notebook


## Data Preprocessing (Replacing URL, Emoji, @username)

In [ ]:
emojiPattern = {':)': 'smile', ':-)': 'smile', ';d': 'wink', ':-E': 'vampire', ':(': 'sad',  ':-(': 'sad', ':-<': 'sad', 
          ':P': 'raspberry', ':O': 'surprised', ':-@': 'shocked', ':@': 'shocked',':-$': 'confused', ':\\': 'annoyed', 
          ':#': 'mute', ':X': 'mute', ':^)': 'smile', ':-&': 'confused', '$_$': 'greedy', '@@': 'eyeroll', ';)': 'wink', 
          ':-!': 'confused', ':-D': 'smile', ':-0': 'yell', 'O.o': 'confused', '<(-_-)>': 'robot', 'd[-_-]b': 'dj', 
          ":'-)": 'sadsmile', ';-)': 'wink', 'O:-)': 'angel','O*-)': 'angel','(:-D': 'gossip', '=^.^=': 'cat'}

urlPattern = r"((http://)[^ ]*|(https://)[^ ]*|( www\.)[^ ]*)"
userPattern = '@[^\s]+'

In [ ]:
new_text = []
for i in range(len(sentimental_df.text)):
    each_tweet = sentimental_df.text.iloc[i].lower()

    modified_tweet = re.sub(urlPattern,' URL', each_tweet)
    for emoji in emojiPattern.keys():
        modified_tweet = modified_tweet.replace(emoji, "EMOJI_" + emojiPattern[emoji])        
    modified_tweet = re.sub(userPattern,' USERNAME', modified_tweet)  
    new_text.append(modified_tweet)

sentimental_df['text'] = pd.Series(new_text)

In [ ]:
sentimental_df.text

0           USERNAME  URL - awww, that's a bummer.  you s...
1          is upset that he can't update his facebook by ...
2           USERNAME i dived many times for the ball. man...
3            my whole body feels itchy and like its on fire 
4           USERNAME no, it's not behaving at all. i'm ma...
                                 ...                        
1599995    just woke up. having no school is the best fee...
1599996    thewdb.com - very cool to hear old walt interv...
1599997    are you ready for your mojo makeover? ask me f...
1599998    happy 38th birthday to my boo of alll time!!! ...
1599999    happy #charitytuesday  USERNAME  USERNAME  USE...
Name: text, Length: 1600000, dtype: object

## Running LTSM Model


In [1]:
from keras.models import Sequential, Model
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from keras.preprocessing.text import Tokenizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from nltk.corpus import stopwords
from keras.layers import *
import keras.backend as kb
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from keras.preprocessing.sequence import pad_sequences
import pandas as pd
import numpy as np

In [ ]:
sentimental_df = pd.read_csv('Sentimental_df_clean.csv')
# X_train_embmean = pd.read_csv('X_train_embmean.csv',header=None).values
# X_val_embmean = pd.read_csv('X_val_embmean.csv',header=None).values
# X_test_embmean = pd.read_csv('X_test_embmean.csv',header=None).values
# text = sentimental_df['text']
# y = sentimental_df['target']


In [ ]:
sentimental_df = sentimental_df[['text','target']]
sentimental_df

,text,target
0,"USERNAME URL - awww, that's a bummer. you s...",0
1,is upset that he can't update his facebook by ...,0
2,USERNAME i dived many times for the ball. man...,0
3,my whole body feels itchy and like its on fire,0
4,"USERNAME no, it's not behaving at all. i'm ma...",0
...,...,...
1599995,just woke up. having no school is the best fee...,1
1599996,thewdb.com - very cool to hear old walt interv...,1
1599997,are you ready for your mojo makeover? ask me f...,1
1599998,happy 38th birthday to my boo of alll time!!! ...,1


In [ ]:
import nltk
nltk.download('stopwords')
def clean(df,stop_words):
  sentimental_df['text'] = sentimental_df['text'].apply(lambda x: ' '.join(x.lower() for x in x.split()))
  sentimental_df['text'] = sentimental_df['text'].apply(lambda x: ' '.join(x for x in x.split() if x not in stop_words))
  return sentimental_df
#  sentimental_df['text'] = sentimental_df['text'].apply(lambda x: [' '.join(x for x in x.split() if x not in stop_words]))
stop_words = stopwords.words('english')
sent_df = clean(sentimental_df,stop_words)


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
text = sent_df['text']
y = sent_df['target']
Y = pd.get_dummies(y).values
tokenizer = Tokenizer(num_words=500, split=' ') 
tokenizer.fit_on_texts(text.values)
X = tokenizer.texts_to_sequences(text.values)
X = pad_sequences(X)

In [ ]:
# from sklearn.model_selection import train_test_split
# X_dev, X_test, y_dev, y_test = train_test_split(X,y, test_size=0.2, random_state=42)
# X_train, X_val, y_train, y_val = train_test_split(X_dev, y_dev, test_size=0.25, random_state=42)

In [ ]:
X_train_embmean.shape

(960000, 100)

In [ ]:
X_train.values

array(['username u sing...you usually sing genre ...whether people accept genre... remains seen',
       'username nuggets redsox lost im going baseball game suprised called hate wake 6:30 ...',
       'username call can, want know whats going on...', ...,
       'disappointed sound quality educamp09 - especially recordings.',
       'finished work good times need rest bball game tomorrow :-d wish luck night yalllll',
       "username how'd get home already?!?!?! nooooo fair!!"],
      dtype=object)

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
model = Sequential()
model.add(Embedding(500, 120, input_length = X.shape[1]))
model.add(SpatialDropout1D(0.4))
model.add(LSTM(176, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(2,activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam', metrics = ['accuracy'])
print(model.summary())

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, 46, 120)           60000     
                                                                 
 spatial_dropout1d_3 (Spatia  (None, 46, 120)          0         
 lDropout1D)                                                     
                                                                 
 lstm_3 (LSTM)               (None, 176)               209088    
                                                                 
 dense_3 (Dense)             (None, 2)                 354       
                                                                 
Total params: 269,442
Trainable params: 269,442
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,Y, test_size=0.2, random_state=42)
#X_train, X_val, y_train, y_val = train_test_split(X_dev, y_dev, test_size=0.25, random_state=42)

In [ ]:
batch_size=256
model.fit(X_train, y_train, epochs = 5, validation_split= 0.25,batch_size=batch_size,validation_steps=100, verbose = 'auto')

Epoch 1/5
3750/3750 [==============================] - 503s 133ms/step - loss: 0.5349 - accuracy: 0.7252 - val_loss: 0.5242 - val_accuracy: 0.7346
Epoch 2/5
3750/3750 [==============================] - 492s 131ms/step - loss: 0.5226 - accuracy: 0.7340 - val_loss: 0.5188 - val_accuracy: 0.7365
Epoch 3/5
3750/3750 [==============================] - 513s 137ms/step - loss: 0.5186 - accuracy: 0.7370 - val_loss: 0.5163 - val_accuracy: 0.7383
Epoch 4/5
3750/3750 [==============================] - 498s 133ms/step - loss: 0.5158 - accuracy: 0.7385 - val_loss: 0.5147 - val_accuracy: 0.7397
Epoch 5/5
3750/3750 [==============================] - 494s 132ms/step - loss: 0.5133 - accuracy: 0.7402 - val_loss: 0.5128 - val_accuracy: 0.7420


In [ ]:
predictions = model.predict(X_test)


In [ ]:
y_pred = np.argmax(predictions,axis=1)
y_test_shaped = np.argmax(y_test,axis=1)

In [ ]:
# 
from sklearn.metrics import roc_auc_score
print('Accuracy Score : ' + str(accuracy_score(y_test_shaped,y_pred)))
print('Precision Score : ' + str(precision_score(y_test_shaped,y_pred)))
print('Recall Score : ' + str(recall_score(y_test_shaped,y_pred)))
print('F1 Score : ' + str(f1_score(y_test_shaped,y_pred)))
print('AUC ROC Score: '+ str(roc_auc_score(y_test_shaped,predictions[:,1])))

Accuracy Score : 0.742203125
Precision Score : 0.7366709342329606
Recall Score : 0.7564203207356734
F1 Score : 0.7464150131411094
AUC ROC Score: 0.8244205036456278


In [ ]:
confusion_matrix(y_test_shaped,y_pred)

array([[107458,  52036],
       [ 42029, 118477]])

In [ ]:



text = sent_df['text']
y = sent_df['target']
Y = pd.get_dummies(y).values
tokenizer = Tokenizer(num_words=500, split=' ') 
tokenizer.fit_on_texts(text.values)
X = tokenizer.texts_to_matrix(text.values,mode='tfidf')
X = pad_sequences(X)

model_ = Sequential()
model_.add(Embedding(500, 120, input_length = X.shape[1]))
model_.add(SpatialDropout1D(0.4))
model_.add(LSTM(176, dropout=0.2, recurrent_dropout=0.2))
model_.add(Dense(2,activation='softmax'))
model_.compile(loss = 'categorical_crossentropy', optimizer='adam', metrics = ['accuracy'])
print(model_.summary())

X_train, X_test, y_train, y_test = train_test_split(X,Y, test_size=0.2, random_state=42)
batch_size=256
model.fit(X_train, y_train, epochs = 5, validation_split= 0.25,batch_size=batch_size,validation_steps=100, verbose = 'auto')



0.8244205036456278

In [ ]:
predictions = model.predict(X_test)
y_pred = np.argmax(predictions,axis=1)
y_test_shaped = np.argmax(y_test,axis=1)

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
cd 'drive/Shareddrives/Applied Machine Learning'

## Rough Code

In [ ]:

def generator_create(batch_size=128):
    input_arr = np.zeros((batch_size,100))
    output_arr = np.zeros((batch_size,2))
    count = 1
    while True:
        for i in range(0,X_train_embmean.shape[0]):
            input_arr[i] = X_train_embmean[i]
            output_arr[i][y_train[i]] = 1
            
            if count == 128:
                temp_sequence = (input_arr,output_arr)
                count = 0
                input_arr = np.zeros((batch_size,100))
                output_arr = np.zeros((batch_size,2))
                yield temp_sequence
            
            count+=1    
                


def develop_model():
    model_1 = Sequential()
    model_1.add(LSTM(128,return_sequences = True,input_shape=(128,100)))
    model_1.add(Dropout(0.2))
    model_1.add(LSTM(24))
    model_1.add(Dropout(0.2))
    model_1.add(Dense(2,activation='softmax'))
    model_1.compile(optimizer = 'adam', loss = 'binary_crossentropy')
    model_1.summary()
    return model_1

def develop_model_without_gen(X_train):
    model_1 = Sequential()
    # model_1.add(LSTM(128))
    model_1.add(Dense(128,input_dim=(X_train.shape[1])))
    # model_1.add(LSTM(128,return_sequences = True))
    # model_1.add(LSTM(128,return_sequences = True,input_shape=(X_train.shape[1],X_train_shape[2])))
    model_1.add(Dropout(0.2))
    model_1.add(Dense(2,activation='softmax'))
    model_1.compile(optimizer = 'adam', loss = 'binary_crossentropy')
    model_1.summary()
    return model_1
    

In [ ]:
# model_1 = Sequential()
# model_1.add(LSTM(128,return_sequences = True,input_shape=(128,100)))
# model_1.add(Dropout(0.2)) 
# model_1.add(Dense(2,activation='softmax'))

# model_1.compile(optimizer = 'adam', loss = 'binary_crossentropy')

# model_1.summary()
# print(model_1.output_shape)

In [ ]:
# batch_size = 128
# MAX_LEN = 100
# generator = generator_create(batch_size)
# steps = len(y_train) * MAX_LEN // batch_size
# model = develop_model()
# model.fit_generator(generator,steps_per_epoch = steps,verbose=True,epochs=5)

In [ ]:
np.array(y_train).reshape(-1,1).shape

(960000, 1)

In [ ]:
Y_train = np.array(y_train)
y_train_one_hot = np.zeros((Y_train.shape[0], Y_train.max()+1))
y_train_one_hot[np.arange(Y_train.size),Y_train] = 1
X_train_reshaped = np.reshape(X_train_embmean, (X_train_embmean.shape[0],1,X_train_embmean.shape[1]))

In [ ]:
X_train_embmean.shape

(960000, 100)

In [ ]:
X_train.shape

(960000,)

In [ ]:
X_train_reshaped.shape

(960000, 1, 100)

In [ ]:
X_train_reshaped.squeeze().shape

(960000, 100)

In [ ]:
model = develop_model_without_gen(X_train_reshaped.squeeze())
model.fit(X_train_reshaped.squeeze(),y_train_one_hot,epochs=5,batch_size=32)

Model: "sequential_31"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_27 (Dense)            (None, 128)               12928     
                                                                 
 dropout_32 (Dropout)        (None, 128)               0         
                                                                 
 dense_28 (Dense)            (None, 2)                 258       
                                                                 
Total params: 13,186
Trainable params: 13,186
Non-trainable params: 0
_________________________________________________________________
Epoch 1/5
30000/30000 [==============================] - 72s 2ms/step - loss: 0.5407
Epoch 2/5
30000/30000 [==============================] - 65s 2ms/step - loss: 0.5377
Epoch 3/5
30000/30000 [==============================] - 65s 2ms/step - loss: 0.5375
Epoch 4/5
30000/30000 [==============================] - 6

In [ ]:
X_train_embmean.shape

(960000, 100)